<a href="https://colab.research.google.com/github/thousandoaks/NVIDIA-RAPIDS/blob/main/Cuxfilter/RAPIDS_CUXFILTER_Example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
!nvidia-smi

Fri Mar 21 12:16:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [69]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [67]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 587 (delta 122), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (587/587), 192.90 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (296/296), done.
Installing RAPIDS remaining 25.02 libraries
Using Python 3.11.11 environment at: /usr
Resolved 173 packages in 11.66s
 Downloaded cuproj-cu12
 Downloaded cuda-bindings
 Downloaded datashader
 Downloaded dask
 Downloaded libkvikio-cu12
 Downloaded ucx-py-cu12
 Downloaded cuspatial-cu12
 Downloaded cucim-cu12
 Downloaded cudf-cu12
 Downloaded rmm-cu12
 Downloaded cuvs-cu12
 Downloaded pylibcugraph-cu12
 Downloaded cugraph-cu12
 Downloaded cuml-cu12
 Downloaded libcuspatial-cu12
 Downloaded pylibcudf-cu12
 Downloaded nvidia-nvcomp-cu12
 Downloaded raft-dask-cu12
 Downloaded libcudf-cu12
Prepared 49 packages in 20.49s
Uninstalled 28 packa

In [80]:
!!pip install jupyter_bokeh
!pip install bokeh_sampledata

In [71]:
import cuxfilter
from cuxfilter import DataFrame, themes, layouts

In [72]:
#update data_dir if you have downloaded datasets elsewhere
DATA_DIR = './data/'

! curl https://data.rapids.ai/viz-data/auto_accidents.arrow.gz --create-dirs -o $DATA_DIR/auto_accidents.arrow.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 50.0M  100 50.0M    0     0   9.7M      0  0:00:05  0:00:05 --:--:-- 10.9M


In [75]:
from cuxfilter.sampledata import datasets_check
datasets_check('auto_accidents', base_dir=DATA_DIR)

Dataset - ./data//auto_accidents.arrow

dataset already downloaded
Extracting ....
Extraction complete


In [76]:
#create cuxfilter DataFrame
cux_df = DataFrame.from_arrow(DATA_DIR + './auto_accidents.arrow')
cux_df.data['ST_CASE'] = cux_df.data['ST_CASE'].astype('float64')

# add a day_week_str column, using cudf.Series.map()
cux_df.data['DAY_WEEK_STR'] = cux_df.data.DAY_WEEK.map({1: 'Sunday',    2: 'Monday',    3: 'Tuesday',    4: 'Wednesday',   5: 'Thursday',    6: 'Friday',    7: 'Saturday',    9: 'Unknown'})

cux_df.data[['dropoff_x', 'dropoff_y', 'DAY_WEEK', 'DAY_WEEK_STR', 'YEAR', 'MONTH']].head()

,dropoff_x,dropoff_y,DAY_WEEK,DAY_WEEK_STR,YEAR,MONTH
0,-9.685585e+06,3.939943e+06,1,Sunday,2017,2
1,-9.661068e+06,4.117979e+06,3,Tuesday,2017,2
2,-9.589649e+06,3.811519e+06,3,Tuesday,2017,1
3,-9.589649e+06,3.811519e+06,3,Tuesday,2017,1
4,-9.589649e+06,3.811519e+06,3,Tuesday,2017,1


In [77]:
type(cux_df.data)

cudf.core.dataframe.DataFrame

In [78]:
demo_red_blue_palette = [ "#3182bd", "#6baed6", "#7b8ed8", "#e26798", "#ff0068" , "#323232" ]

chart1 = cuxfilter.charts.scatter(x='dropoff_x', y='dropoff_y', aggregate_col='DAY_WEEK', aggregate_fn='mean',
                                color_palette=demo_red_blue_palette, tile_provider='CartoLight',
                                pixel_shade_type='linear')
chart2 = cuxfilter.charts.bar('YEAR')


chart3 = cuxfilter.charts.multi_select('DAY_WEEK_STR')

chart4 = cuxfilter.charts.number(expression="AGE", aggregate_fn="mean", title="Mean age")
chart5 = cuxfilter.charts.number(expression="SIDE_DRIV_STARS + FRNT_DRIV_STARS", aggregate_fn="mean", title="Vehicle(Mean front+side safety rating)")
charts_list = [chart1, chart2 ]
sidebar = [chart3, chart4, chart5]

In [81]:
d = cux_df.dashboard(charts_list, sidebar=sidebar, title='Auto Accidents Dashboard', layout=layouts.feature_and_base, theme=themes.default)

In [82]:
"""
1. d.show('current_notebook_url:current_notebook_port', sidebar_width=300, height=1000) remote dashboard

2. d.app() inline within the notebook cell

Incase you need to stop the server:

- d.stop()
"""

# uncomment the line below to run the dashboard inline within the notebook cell
d.app(width=1000, height=800)

GridSpec(ncols=12, nrows=5)
    [0] GridSpec(height=800, ncols=12, nrows=5, sizing_mode='fixed', width=1000)
        [0] HoloViews(DynamicMap, height=480, sizing_mode='stretch_both', width=1000)
        [1] HoloViews(DynamicMap, height=320, sizing_mode='stretch_both', width=1000)
    [1] WidgetBox(styles={'border-color': '...})
        [0] Number(css_classes=['indicator'], default_color='#2B2B2B', font_size='18pt', format='{value:,}', name='Datapoints Selected', sizing_mode='stretch_width', title_size='14pt', value=1296221)
        [1] Progress(sizing_mode='stretch_width', styles={'--success-bg-color': '...}, value=100)
    [2] Column(min_height=500, sizing_mode='stretch_width')
        [0] MultiChoice(name='DAY_WEEK_STR', options=['Sunday', 'Tuesday', ...], sizing_mode='scale_width', styles={'color': '#2B2B2B'}, stylesheets=['\n                .choic...])
        [1] Card(active_header_background='#ffffff', collapsible=False, css_classes=['card', 'number-card'], header='Mean age', sizing_mode='scale_width', styles={'background-color': '#fff...})
            [0] Number(css_classes=['number-card-value'], font_size='18pt', value=57)
        [2] Card(active_header_background='#ffffff', collapsible=False, css_classes=['card', 'number-card'], header='Vehicle(Mean f..., sizing_mode='scale_width', styles={'background-color': '#fff...})
            [0] Number(css_classes=['number-card-value'], font_size='18pt', value=2)